In [ ]:
pip install google-cloud-batch ipywidgets

### Download sample data

In [ ]:
!wget -O parabricks_sample.tar.gz     "https://s3.amazonaws.com/parabricks.sample/parabricks_sample.tar.gz"

In [ ]:
!tar -xvf parabricks_sample.tar.gz

In [1]:
import csv
from ipywidgets import interact, widgets

## Read CSV file and generate batch job variables

In [3]:
input_csv = "fq_list.csv"

In [ ]:
batch_job_desc = """
{
  "logsPolicy": {
    "destination": "CLOUD_LOGGING"
  },
  "taskGroups": [
    {
      "taskSpec": {
        "runnables": [
          {
            "script": {
              "text": "gcloud storage cp $INPUT_PATH /tmp/file_to_be_processed" // copy the file from GCS to local
            }
          },
          {
              "container": {
              "imageUri": "nvcr.io/nvidia/clara/clara-parabricks:4.1.0-1",
              "commands": [
                "-c",
                "pbrun fq2bam --ref Ref/Homo_sapiens_assembly38.fasta \
--in-fq Data/sample_1.fq.gz Data/sample_2.fq.gz  \
--knownSites Ref/Homo_sapiens_assembly38.known_indels.vcf.gz \
--out-bam mark_dups_gpu.bam \
--out-recal-file recal_gpu.txt \
--tmp-dir /raid/myrun"
              ],
              "entrypoint": "/bin/sh"
            }
          }
        ]
        "taskCount": "3",
        "taskCountPerNode": "1",
      },
      "taskEnvironments": [
        {
          "variables": {
            "INPUT_PATH": "input_path_0"
          }
        },
        {
          "variables": {
            "INPUT_PATH": "input_path_2"
          }
        },
        {
          "variables": {
            "INPUT_PATH": "input_path_3"
          }
        }
      ]
    }
  ]
}
"""